In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local").appName("spark_sql_study").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/17 15:41:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
datas = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'),
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USA', 1222, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]

In [4]:
datasSchema = ["name", "ticker", "country", "price", "currency"]

In [5]:
# Schema에 데이터 타입을 지정하지 않으면 spark에서 자동으로 지정
df = spark.createDataFrame(data=datas, schema=datasSchema)
df.dtypes

[('name', 'string'),
 ('ticker', 'string'),
 ('country', 'string'),
 ('price', 'bigint'),
 ('currency', 'string')]

In [11]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [12]:
# DataFrame을 데이터베이스처럼 SQL 문을 사용하려면 temporary view를 만들어야 함!!
df.createOrReplaceTempView("stocks")

In [14]:
spark.sql("SELECT * FROM stocks").show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [15]:
spark.sql("SELECT name, ticker FROM stocks WHERE country = 'USA'").show()

+-------+------+
|   name|ticker|
+-------+------+
| Google| GOOGL|
|Netflix|  NFLX|
| Amazon|  AMZN|
|  Tesla|  TSLA|
+-------+------+



In [16]:
spark.sql("SELECT name, price FROM stocks WHERE price < 2000").show()

+-------+-----+
|   name|price|
+-------+-----+
|Netflix|  645|
|  Tesla| 1222|
|Tencent|  483|
+-------+-----+



In [18]:
spark.sql("SELECT * FROM stocks WHERE name like '%o%' and price > 2000").show()

+------+------+-------+------+--------+
|  name|ticker|country| price|currency|
+------+------+-------+------+--------+
|Google| GOOGL|    USA|  2984|     USD|
|Amazon|  AMZN|    USA|  3518|     USD|
|Toyota|  7203|  Japan|  2006|     JPY|
| Kakao|035720|  Korea|125000|     KRW|
+------+------+-------+------+--------+



In [19]:
spark.sql("SELECT * FROM stocks WHERE name like '%o%' and price between 2500 and 5000").show()

+------+------+-------+-----+--------+
|  name|ticker|country|price|currency|
+------+------+-------+-----+--------+
|Google| GOOGL|    USA| 2984|     USD|
|Amazon|  AMZN|    USA| 3518|     USD|
+------+------+-------+-----+--------+



In [20]:
# Tesla 보다 높은 가격을 가진 화폐단위 USD 를 쓰는 곳
spark.sql("SELECT * FROM stocks \
 WHERE currency = 'USD' AND \
 price > (SELECT price FROM stocks WHERE name = 'Tesla')").show()

+------+------+-------+-----+--------+
|  name|ticker|country|price|currency|
+------+------+-------+-----+--------+
|Google| GOOGL|    USA| 2984|     USD|
|Amazon|  AMZN|    USA| 3518|     USD|
+------+------+-------+-----+--------+



In [21]:
spark.sql("SELECT * FROM stocks WHERE name LIKE '%o%' AND price > 2000 ORDER BY price asc").show()

+------+------+-------+------+--------+
|  name|ticker|country| price|currency|
+------+------+-------+------+--------+
|Toyota|  7203|  Japan|  2006|     JPY|
|Google| GOOGL|    USA|  2984|     USD|
|Amazon|  AMZN|    USA|  3518|     USD|
| Kakao|035720|  Korea|125000|     KRW|
+------+------+-------+------+--------+



In [22]:
spark.sql("SELECT * FROM stocks WHERE name LIKE '%o%' AND price > 2000 ORDER BY price desc").show()

+------+------+-------+------+--------+
|  name|ticker|country| price|currency|
+------+------+-------+------+--------+
| Kakao|035720|  Korea|125000|     KRW|
|Amazon|  AMZN|    USA|  3518|     USD|
|Google| GOOGL|    USA|  2984|     USD|
|Toyota|  7203|  Japan|  2006|     JPY|
+------+------+-------+------+--------+



In [23]:
earnings = [
    ('Google', 27.99, 'USD'),
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW')
]

In [25]:
from pyspark.sql.types import StringType, FloatType, StructField, StructType

In [26]:
earningsSchema = StructType([
    StructField("name", StringType(), True),
    StructField("eps", FloatType(), True),
    StructField("currency", StringType(), True),
])

In [27]:
earningsDF = spark.createDataFrame(data=earnings, schema=earningsSchema)
earningsDF.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [28]:
earningsDF.createOrReplaceTempView("earnings")

In [29]:
earningsDF.select("*").show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [30]:
spark.sql("SELECT * FROM stocks join earnings on stocks.name = earnings.name").show()

+-------+------+---------+------+--------+-------+------+--------+
|   name|ticker|  country| price|currency|   name|   eps|currency|
+-------+------+---------+------+--------+-------+------+--------+
| Amazon|  AMZN|      USA|  3518|     USD| Amazon|  6.12|     USD|
| Google| GOOGL|      USA|  2984|     USD| Google| 27.99|     USD|
|  Kakao|035720|    Korea|125000|     KRW|  Kakao| 705.0|     KRW|
|Netflix|  NFLX|      USA|   645|     USD|Netflix|  2.56|     USD|
|Samsung|005930|    Korea| 70600|     KRW|Samsung|1780.0|     KRW|
|Tencent|  0700|Hong Kong|   483|     HKD|Tencent| 11.01|     HKD|
|  Tesla|  TSLA|      USA|  1222|     USD|  Tesla|  1.86|     USD|
| Toyota|  7203|    Japan|  2006|     JPY| Toyota|224.82|     JPY|
+-------+------+---------+------+--------+-------+------+--------+



In [31]:
spark.stop()